# <b>DNaseI-Seq workshop part 1 - Read alignment and identification of open chromatin regions</b>

In this practical we will be processing two DNaseI-Seq samples obtained from mouse T-Cells

Here we have two types of T-Cells (denoted as NAg and TAg) which are:
* NAg - Naive T-Cells which have been stimulted with antigen
* TAg - Tolerant T-Cells which have been stimulated by antigen

By comparing the DHS profiles of these two samples we can show that the epigenetic response to stimulation with antigen of these two cell types is altered

This data is published and is described in more detail in:

Bevington et al. (2020). Chromatin Priming Renders T Cell Tolerance-Associated Genes Sensitive to Activation below the Signaling Threshold for Immune Response Genes. <i>Cell Reports</i>. 31(10), 107748

* In order to save time we will only be processing data from mouse chromosome 1
* In part 2 of this workshop you will work with processed data from these samples that contain data for the entire genome as well as additional bioloical replicates

## <b>Some useful Linux commands</b>

In [ ]:
ls # List directory - this shows what is in your folder
mkdir # Make directory - Create a new folder
cd # Change directory - Move into a directory
rm # Remove file - Deletes a file - Be careful, Linux has no recycle bin!!

## <b>Step 0. Preparation</b>

Load the required software into your bluebear session

In [ ]:
module load FastQC/0.11.9-Java-11
module load Trimmomatic/0.39-Java-11
module load Bowtie2/2.3.5.1-GCC-8.3.0
module load SAMtools/1.10-GCC-8.3.0
module load picard/2.21.1-Java-11
module load MACS2/2.2.7.1-foss-2019b-Python-3.7.4
module load BEDTools/2.29.2-GCC-8.3.0
module load Subread/2.0.1-GCC-8.3.0

* Move to the folder that we have set up for this class
* Create your own folder and copy the data for this workshop to it

In [ ]:
# Move to workshop folder
cd /rds/projects/k/knowletj-mibtp-masterclasses/MITBP_Masterclass_2021

# Create your own folder and go into it
mkdir YOUR_NAME
cd YOUR_NAME

# Copy the fastq files to your folder
cp ../data_for_workshop/fastq/* .

# Show the files have been copied correctly
ls

## <b>Step 1. Quality assessment of raw reads and read trimming</b>

To inspect the read quality we will use a program called <b>fastqc</b>\
Run this command for both samples

In [ ]:
fastqc NAg_Rep1_sample.fastq.gz
fastqc TAg_Rep1_sample.fastq.gz

This creates a html report thatcan be opened in your web browser

While the quality of these reads looks good, we will still carry out tread trimming to try improve it further\
This trimming will carry out a number of steps:
1. Removal of sequencing adaptors
2. Removal of low-quality bases from the 3-prime end of the sequence reads (which could contain sequencng errors)
3. Any read that is too short after trimming is removed from the fastq file

To do this we will use a tool called <b>trimmomatic</b>

In [ ]:
# First, set the path to the sequencing adaptors
# These are stored in a fastA formatted text file
ADAPTORS=/rds/projects/k/knowletj-mibtp-masterclasses/MITBP_Masterclass_2021/data_for_workshop/Annotation/Trimmomatic_adapters/TruSeq3-SE.fa

# Now run the software
java -jar $EBROOTTRIMMOMATIC/trimmomatic-0.39.jar SE NAg_Rep1_sample.fastq.gz NAg_Rep1_sample_trimmed.fastq.gz ILLUMINACLIP:$ADAPTORS:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:20 MINLEN:50
java -jar $EBROOTTRIMMOMATIC/trimmomatic-0.39.jar SE TAg_Rep1_sample.fastq.gz TAg_Rep1_sample_trimmed.fastq.gz ILLUMINACLIP:$ADAPTORS:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:20 MINLEN:50

Lets have a look at the different parameters of these commands

* <span style="color:#0000DD">java -jar $EBROOTTRIMMOMATIC/trimmomatic-0.39.jar SE</span> - This runs the software in single-end mode. If you have paired-end data you should instead specify PE
* <span style="color:#0000DD">NAg_Rep1_sample.fastq.gz</span> - This is the input file
* <span style="color:#0000DD">NAg_Rep1_sample_trimmed.fastq.gz</span> - This is the output file
* <span style="color:#0000DD">ILLUMINACLIP:\$ADAPTORS:2:30:10</span> - This provides the path to the fasta file with the sequence adaptors. The numbers control how many mis-matches between the sequence read and the adaptor is acceptable to still find a match
* <span style="color:#0000DD">LEADING:3 TRAILING:3</span> - Remove bases with a quality score less than 3 from the beginning and end of the read
* <span style="color:#0000DD">SLIDINGWINDOW:4:20</span> - Trim reads from the end of the read until the average base quality in a 4bp window is above 20
* <span style="color:#0000DD">MINLEN:50</span> - Remove reads that are less than 50 bases long after trimming

For more information on these go to http://www.usadellab.org/cms/?page=trimmomatic

## <b>Step 2. Read alignment and PCR duplicate removal</b>

* The next step is to align the reads to the genome.
* Here we will only align data to mouse chromosome 1 (mouse genome version mm10)

* To do the alignment we will be using <b>bowtie2</b>
* bowtie2 requires an indexed genome to align the data to
* This has already been created for you so no need to do this here. But for reference a genome index can be created from the genome fastA file like so:
bowtie2-build mm10_chr1.fa mm10

* Genome fasta files can be obtained from ensembl or the UCSC genome browser

Next, align the data with bowtie2 - the results will be stores in the sequence alignment map (SAM) format

In [ ]:
# Align the NAg sample
bowtie2 --very-sensitive-local -x /rds/projects/k/knowletj-mibtp-masterclasses/MITBP_Masterclass_2021/data_for_workshop/Genome/mm10_chr1 -U NAg_Rep1_sample_trimmed.fastq.gz -S NAg_Rep1_sample.sam

# Align the TAg sample
bowtie2 --very-sensitive-local -x /rds/projects/k/knowletj-mibtp-masterclasses/MITBP_Masterclass_2021/data_for_workshop/Genome/mm10_chr1 -U TAg_Rep1_sample_trimmed.fastq.gz -S TAg_Rep1_sample.sam

* <span style="color:#0000DD">--very-sensitive-local</span> - Controls how hard bowtie will work to find an alignment for a read. This helps improve alignment quality\
* <span style="color:#0000DD">-x</span> - Path to the genome index files\
* <span style="color:#0000DD">-U</span> - The reads to be aligned in fastq format. These are unpaired reads to we use the -U flag. For paired-end we would supply two files with -1 <left_reads.fastq> -2 <right_reads.fastq>\
* <span style="color:#0000DD">-S</span> - The output file in SAM format

* Before we move to the next step, the aligned reads need to be sorted by chromosome and position
* We will also convert the files into the binary alignment map (BAM) format
* We can do this using <b>samtools</b>

In [ ]:
samtools sort -o NAg_Rep1_sample.bam NAg_Rep1_sample.sam
samtools sort -o TAg_Rep1_sample.bam TAg_Rep1_sample.sam

* During the library prep step the fragments to be sequenced were amplified using PCR
* As a result, we might find that we have multiple identical reads that actually represent the same DNA fragment
* We may want to remove these to avoid any potential artefacts from over-amplified fragments
* <b>Picard Tools</b> is useful for this

In [ ]:
java -jar $EBROOTPICARD/picard.jar MarkDuplicates I=NAg_Rep1_sample.bam O=NAg_Rep1_sample_rmdup.bam M=NAg_Rep1_sample_picard_metrics.txt AS=true REMOVE_DUPLICATES=true
java -jar $EBROOTPICARD/picard.jar MarkDuplicates I=TAg_Rep1_sample.bam O=TAg_Rep1_sample_rmdup.bam M=TAg_Rep1_sample_picard_metrics.txt AS=true REMOVE_DUPLICATES=true

Here is what each of these options do:
* <span style="color:#0000DD">I=NAg_Rep1_sample.bam</span> - The input file which is our sorted bam file 
* <span style="color:#0000DD">O=NAg_Rep1_sample_rmdup.bam</span> - The output file
* <span style="color:#0000DD">M=NAg_Rep1_sample_picard_metrics.txt</span> - Ouput some metrics about the duplicate removal
* <span style="color:#0000DD">AS=true</span> - Assume the reads are sorted
* <span style="color:#0000DD">REMOVE_DUPLICATES=true</span> - Remove PCR duplicated reads

We also need to create an index for the aligned reads

In [ ]:
samtools index NAg_Rep1_sample_rmdup.bam
samtools index TAg_Rep1_sample_rmdup.bam

## <b>Step 3. Peak calling and filtering peaks</b>

* The next step will be to identify potential open chromatin regions - which we can also call peaks
* To do this we will use <b>MACS2</b>

In [ ]:
macs2 callpeak -t NAg_Rep1_sample_rmdup.bam -n NAg_Rep1_sample --keep-dup all --nomodel -g mm -B --trackline
macs2 callpeak -t TAg_Rep1_sample_rmdup.bam -n TAg_Rep1_sample --keep-dup all --nomodel -g mm -B --trackline

* <span style="color:#0000DD">-t NAg_Rep1_sample_rmdup.bam</span> - The aligned reads from the treatment file - in this case this is sorted, de-duplicated alignment that we have made\
* <span style="color:#0000DD">-n NAg_Rep1_sample</span> - What to call the output files\
* <span style="color:#0000DD">--keep-dup all</span> - Tells macs not to perform PCR duplicate removal - we have already done this\
* <span style="color:#0000DD">--nomodel</span> - Turns off MAC's peak shiting model. This model is useful for ChIP-Seq but for other types of data like DNaseI of ATAC-Seq we do not need this\
* <span style="color:#0000DD">-g mm</span> - This specifies the effective genome size. MACS2 has some genome sizes for common model organisms built in built-in. Here we specific mouse (mus musculus) using the abbreviation mm\
* <span style="color:#0000DD">-B --trackline</span> - These options are used to produce a bedGraph file. This file can be uploaded to the UCSC genome browser

This produces a number of output files - They are:
* <b>NAg_Rep1_sample_peaks.narrowPeak</b> - A BED file that contains the peak positions called by MACS2 along with p-value and q-value of those peaks
* <b>NAg_Rep1_sample_summits.bed</b> - A BED file with the summit position for every peak called
* <b>NAg_Rep1_sample_peaks.xls</b> - A tab-delimited file which contains the peak and summit positions as well as some other useful statistics (peak height, pvalue, qvalue)
* <b>NAg_Rep1_sample_control_lambda.bdg</b> - BedGraph for the input sample (used only with chip)
* <b>NAg_Rep1_sample_treat_pileup.bdg</b> - BedGraph for the treatment sample (can be uploaded to UCSC genome browser)

Next, count the number of peaks called by MACS2

In [ ]:
wc -l NAg_Rep1_sample_summits.bed
wc -l TAg_Rep1_sample_summits.bed

Next we will filter the peaks. We will do this in two steps:
1. Remove small background peaks
2. Remove blacklisted regions

The below set of commands use the linux pipe to reads the peaks.xls file produced by macs and extract peaks with a height >= 10

In [ ]:
cat NAg_Rep1_sample_peaks.xls | grep -v '#' | grep -v 'start' | awk '$6 >= 10' | awk '{print $1"\t"$5"\t"$5}' > NAg_Rep1_sample_filtered_summits.bed
cat TAg_Rep1_sample_peaks.xls | grep -v '#' | grep -v 'start' | awk '$6 >= 10' | awk '{print $1"\t"$5"\t"$5}' > TAg_Rep1_sample_filtered_summits.bed

This is what each of these commands is doing:
* <span style="color:#0000DD">cat NAg_Rep1_sample_peaks.xls</span> - This reads the file
* <span style="color:#0000DD">grep -v '#' | grep -v 'start'</span> - This removes the headers from the file (i.e. lines that begin with # or contain the word start)
* <span style="color:#0000DD">awk '$6 >= 10'</span> - Keep only peaks with a value >= 10 in column 6
* <span style="color:#0000DD">awk '{print $1"\t"$5"\t$5}'</span> - Keep only the 1st column (which contains the chromosome) and the 5th column (summit position) twice - this ensures we have BED formatted file
* <span style="color:#0000DD">> NAg_Rep1_sample_filtered_summits.bed</span> - The output file - A BED file

Count the number of peaks kept after filtering

In [ ]:
wc -l NAg_Rep1_sample_filtered_summits.bed
wc -l TAg_Rep1_sample_filtered_summits.bed

* Next we will remove blacklisted regions
* These are regions which are known to give unusually high read coverage and are considered an artefact
* The mm10 blacklist can be found here as a bed file:
* It has already been downladed for you so you can do the filtering as so

In [ ]:
# Set the path to the blacklist bed file
BLACKLIST=/rds/projects/k/knowletj-mibtp-masterclasses/MITBP_Masterclass_2021/data_for_workshop/Annotation/mm10-blacklist.v2.bed

bedtools intersect -v -a NAg_Rep1_sample_filtered_summits.bed -b $BLACKLIST > NAg_Rep1_sample_filtered_noBlackList_summits.bed
bedtools intersect -v -a TAg_Rep1_sample_filtered_summits.bed -b $BLACKLIST > TAg_Rep1_sample_filtered_noBlackList_summits.bed

Here, we used the <b>intersect</b> function from <b>bedtools</b>
The command can be broken down like so:
* <span style="color:#0000DD">-a NAg_Rep1_sample_filtered_summits.bed</span> - This is bed file A - Contains our filtered peaks
* <span style="color:#0000DD">-b \$BLACKLIST</span> - This is bed file B - A bed file of blacklisted peaks to remove
* <span style="color:#0000DD">-v</span> - This option tells bedtools to only keep sites from file A that are not in file B
* <span style="color:#0000DD">>NAg_Rep1_sample_filtered_noBlackList_summits.bed</span> - The output file

Count the number of peaks after removing blacklisted sites

In [ ]:
wc -l NAg_Rep1_sample_filtered_noBlackList_summits.bed
wc -l TAg_Rep1_sample_filtered_noBlackList_summits.bed

## <b>Step 4. Create a peak union and count reads</b>

Before we can create a table of read counts for the differential peak analysis, we must first combine all the peaks from our samples into a single file\
We refer to this a the peak union

Our criteria for merging peaks is as follows:
* If the summit positions of two peaks is within 200bp we consider them to be the same peak and merge them


To do this we will first need to do two things:
1. Extend the peaks by +/- 100bp to create a 200bp peak window. This is done using the <b>slop</b> function in <b>bedtools</b>
2. Concatenate all peaks into one single bed file. This file then has to be sorted by chromosome and position. For this we will use the <b>cat</b> function in Linux and the <b>sort</b> function in <b>bedtools</b>

Finally we can merge overlapping peaks using the <b>merge</b> function in <b>bedtools</b>

In [ ]:
# For bedtools slop we need a table of chromosome sizes. Set the path to this table like so
CHROMSIZE=/rds/projects/k/knowletj-mibtp-masterclasses/MITBP_Masterclass_2021/data_for_workshop/Annotation/mm10.chrom.sizes

# Now run bedtools slop
bedtools slop -b 100 -i NAg_Rep1_sample_filtered_noBlackList_summits.bed -g $CHROMSIZE > NAg_Rep1_sample_filtered_noBlackList_peaks.bed
bedtools slop -b 100 -i TAg_Rep1_sample_filtered_noBlackList_summits.bed -g $CHROMSIZE > TAg_Rep1_sample_filtered_noBlackList_peaks.bed

Here is what these options do:
* <span style="color:#0000DD">-b 100</span> - extend summit by 100bp in both directions
* <span style="color:#0000DD">-i NAg_Rep1_sample_filtered_noBlackList_summits.bed</span> - The input file
* <span style="color:#0000DD">-g mm10.chromSizes</span> - A file that contains the size (in base pairs) of each of the chromosomes in mm10
* <span style="color:#0000DD">> NAg_Rep1_sample_filtered_noBlackList_peaks.bed</span> - The output file

Next, we can combine the two bed files in to a single file
At this point we also need to sort the peaks and then merge peaks which are overlapping

In [ ]:
cat NAg_Rep1_sample_filtered_noBlackList_peaks.bed TAg_Rep1_sample_filtered_noBlackList_peaks.bed > Merged.bed
bedtools sort -i Merged.bed > Merged_sorted.bed
bedtools merge -i Merged_sorted.bed > Peak_Union.bed

Count the number of peaks in the peak union

In [ ]:
wc -l Peak_Union.bed

Finally, we can count the number of reads per peak using <b>featureCounts</b>. 
This is what we will use for the differential peak analysis

featureCounts will not read BED files directly, so first we need to convert the peak union bed file into something featureCounts can read \
We will use the SAF file format here, which is extremely similar to a bed file with two extra columns

For this we can use <b>awk</b>

In [ ]:
cat Peak_Union.bed | awk '{print $1":"$2":"$3"\t"$1"\t"$2"\t"$3"\t."}' > Peak_Union.saf

This will create a file that looks something like:

| PeakID | Chromosome | Start | End | Strand |
| --- | --- | --- | --- | --- |
| chr7\:110166237:110166637 | chr7 | 110166237 | 110166637 | . |
| chr17\:79298879:79299279 | chr17 | 79298879 | 79299279 | . |
| chr19\:16789975:16790375 | chr19 | 16789975 | 16790375 | . |

* The first column is just the peak coordinate separated by :
* The next 3 columns are the peak position in a BED like format
* The final column is the strand - since this is genomic data we do not actually have stranded data so a . will work fine here

The column headers can be left out

Now, we can count reads using <b>featureCounts<b>

In [ ]:
featureCounts -F SAF -a Peak_Union.saf -o NAgTAg_DHS_rawCounts.tsv NAg_Rep1_sample_rmdup.bam TAg_Rep1_sample_rmdup.bam

Here is what these options do:
* <span style="color:#0000DD">-F SAF</span> - tells featureCounts our peaks are in the SAF format
* <span style="color:#0000DD">-a Peak_Union.saf</span> - The input file
* <span style="color:#0000DD">-o NAgTAg-DHS_rawCounts.tsv</span> - The output file
* <span style="color:#0000DD">NAg_Rep1_sample_rmdup.bam TAg_Rep1_sample_rmdup.bam</span> - Here we provide the bam files we made earlier - use these to count the reads

The resulting file can be used for downstream analysis in DESeq2 or other programs

# End of workshop part 1. Move to part 2